# __Scrape Historical Fight Data__

### Import Libraries

In [3]:
import requests
import pandas as pd
import string
import IPython
import time
from bs4 import BeautifulSoup
from pandas.plotting import scatter_matrix
import os 
import re
from dateutil import parser

%matplotlib inline

/anaconda3/envs/fight_prediction/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/envs/fight_prediction/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


### Get links for each UFC event

In [4]:
all_events_url = "http://ufcstats.com/statistics/events/completed?page=all"
all_events_page = requests.get(all_events_url)
all_events_soup = BeautifulSoup(all_events_page.content, 'html.parser')

In [5]:
table = all_events_soup.find('table')
events = table.findAll('a')
# Pop the first link because it is for the next UFC event
events.pop(0,)

<a class="b-link b-link_style_white" href="http://ufcstats.com/event-details/b5882371e2a3900d">
                          UFC 234: Whittaker vs. Gastelum
                        </a>

### Create Dictionary for Statistics

In [6]:
%%time
d = {}
# Make a for loop with dictionary to get a bunch of these thigngs. 
# Create empty lists for fighters and winner
fighters_1 = []
fighters_2 = []
winners = []
# General information
dates = []
fight_urls = []

num_rounds = 5
cur_round = 1
while(cur_round <= num_rounds):
    fighter_index = 1
    while fighter_index <= 2:
        
        d["f{0}_r{1}_kds".format(fighter_index, cur_round)] = []
        d["f{0}_r{1}_sigstrikes_l".format(fighter_index, cur_round)] = []
        d["f{0}_r{1}_sigstrikes_l".format(fighter_index, cur_round)] = []
        d["f{0}_r{1}_sigstrikes_a".format(fighter_index, cur_round)] = []
        d["f{0}_r{1}_strikes_l".format(fighter_index, cur_round)] = []
        d["f{0}_r{1}_strikes_a".format(fighter_index, cur_round)] = []
        d["f{0}_r{1}_tds_l".format(fighter_index, cur_round)] = []
        d["f{0}_r{1}_tds_a".format(fighter_index, cur_round)] = []
        d["f{0}_r{1}_subs_a".format(fighter_index, cur_round)] = []
        d["f{0}_r{1}_passes".format(fighter_index, cur_round)] = []
        d["f{0}_r{1}_revs".format(fighter_index, cur_round)] = []
        
        fighter_index += 1
    cur_round += 1

CPU times: user 78 µs, sys: 0 ns, total: 78 µs
Wall time: 80.8 µs


In [7]:
%%time
page_count = 1

num_errors = 0
for event in events:
    try:
        # Setup  url, page, and soup for EVENT
        event_url = event['href']
        event_page = requests.get(event_url)
        event_soup = BeautifulSoup(event_page.content, 'html.parser')
        
        # Extract date
        date_text = event_soup.find("li",{"class":"b-list__box-list-item"}).text.strip().strip("Date:").strip("\n").strip()
        date = parser.parse(date_text)
        
        # Parse table for statistics
        table = event_soup.find('table')
        rows = table.findAll('tr',onclick=True)
        for row in rows:
            # Setup FIGHT url, page, and soup for FIGHT
            fight_url = row['onclick'][7:-2]
            # ALTER = Might need to add a sleep timer
            fight_page = requests.get(fight_url)
            fight_soup = BeautifulSoup(fight_page.content, 'html.parser')
            
            # Add fighters
            section = fight_soup.findAll("div",{"class":"b-fight-details__person"})
            
            winner = ''
            for i,fighter in enumerate(section,1):
                if i == 1:
                    fighter_1 = ' '.join(fighter.text.replace('\n','').strip().split()[1:])
                    
                elif i == 2:
                    fighter_2 = ' '.join(fighter.text.replace('\n','').strip().split()[1:])
                    
                # Add winner
                if(fighter.text.strip()[0] == "W"):
                    winner = ' '.join(section[0].text.replace('\n','').strip().split()[1:])
    
            if winner == '':
                winner = 'No Winner'
                
            # START HERE
            stats_tables = fight_soup.findAll("table",{"class","b-fight-details__table js-fight-table"}) ## Start here
            round_stats_table = stats_tables[0]
            p_r_stats = round_stats_table.findAll('p',{'class':"b-fight-details__table-text"})
            
            
            # Create Indices
            stat_ind = 2
            round_ind = 1
            
            while(round_ind <= 5):
                fighter_ind = 1
                while fighter_ind <= 2:
                    # Try except is used because many of the fights don't go full the distance
                    try: 
                        # Scrape Knockdowns
                        d['f{0}_r{1}_kds'.format(fighter_ind, round_ind)].append(p_r_stats[stat_ind].text.replace('\n','').strip())
                        stat_ind += 2
                        # Scrape Significant Strikes
                        d['f{0}_r{1}_sigstrikes_l'.format(fighter_ind, round_ind)].append(p_r_stats[stat_ind].text.replace('\n','').strip().split()[0]) 
                        d['f{0}_r{1}_sigstrikes_a'.format(fighter_ind, round_ind)].append(p_r_stats[stat_ind].text.replace('\n','').strip().split()[2]) 
                        # Scrape Strikes
                        stat_ind += 4
                        d['f{0}_r{1}_strikes_l'.format(fighter_ind, round_ind)].append(p_r_stats[stat_ind].text.replace('\n','').strip().split()[0]) 
                        d['f{0}_r{1}_strikes_a'.format(fighter_ind, round_ind)].append(p_r_stats[stat_ind].text.replace('\n','').strip().split()[2])
                        # Scrape Takedowns
                        stat_ind += 2
                        d['f{0}_r{1}_tds_l'.format(fighter_ind, round_ind)].append(p_r_stats[stat_ind].text.replace('\n','').strip().split()[0]) 
                        d['f{0}_r{1}_tds_a'.format(fighter_ind, round_ind)].append(p_r_stats[stat_ind].text.replace('\n','').strip().split()[2])
                        # Scrape Subs
                        stat_ind += 4
                        d['f{0}_r{1}_subs_a'.format(fighter_ind, round_ind)].append(p_r_stats[stat_ind].text.replace('\n','').strip())
                        # Scrape Pass
                        stat_ind += 2
                        d['f{0}_r{1}_passes'.format(fighter_ind, round_ind)].append(p_r_stats[stat_ind].text.replace('\n','').strip())  
                        # Scrape Rev.
                        stat_ind += 2
                        d['f{0}_r{1}_revs'.format(fighter_ind, round_ind)].append(p_r_stats[stat_ind].text.replace('\n','').strip())
                    except:
                        d['f{0}_r{1}_kds'.format(fighter_ind, round_ind)].append(None)
                        d['f{0}_r{1}_sigstrikes_l'.format(fighter_ind, round_ind)].append(None)
                        d['f{0}_r{1}_sigstrikes_a'.format(fighter_ind, round_ind)].append(None)
                        d['f{0}_r{1}_strikes_l'.format(fighter_ind, round_ind)].append(None)
                        d['f{0}_r{1}_strikes_a'.format(fighter_ind, round_ind)].append(None)
                        d['f{0}_r{1}_tds_l'.format(fighter_ind, round_ind)].append(None)
                        d['f{0}_r{1}_tds_a'.format(fighter_ind, round_ind)].append(None)
                        d['f{0}_r{1}_subs_a'.format(fighter_ind, round_ind)].append(None)
                        d['f{0}_r{1}_passes'.format(fighter_ind, round_ind)].append(None)
                        d['f{0}_r{1}_revs'.format(fighter_ind, round_ind)].append(None)  
                                                                                  
                    
                    if fighter_ind == 1:             
                        stat_ind = 3+20*(round_ind-1)
                    elif fighter_ind == 2: 
                        stat_ind = 2+20*(round_ind -1)
                    
                    fighter_ind +=1
                    
                round_ind += 1
            
            # Append data to lists
            fighters_1.append(fighter_1)
            fighters_2.append(fighter_2)
            winners.append(winner)
            fight_urls.append(fight_url)
            dates.append(date)
    except:
        num_errors += 1
        print("There are " + num_errors)
        
    page_count +=1
    
    if page_count == 3:
        break

CPU times: user 1.36 s, sys: 41.4 ms, total: 1.4 s
Wall time: 1min 40s


In [8]:
round_stats_df = pd.DataFrame(d)

gen_stats_dict = {'fighter_1':fighters_1,
              'fighter_2':fighters_2,
              'winner':winners,           
              'url': fight_urls,
              'date': dates}

gen_stats_df = pd.DataFrame(gen_stats_dict, columns=gen_stats_dict.keys())

all_stats_df = pd.concat([gen_stats_df, round_stats_df], axis=1)
all_stats_df

,fighter_1,fighter_2,winner,url,date,f1_r1_kds,f1_r1_sigstrikes_l,f1_r1_sigstrikes_a,f1_r1_strikes_l,f1_r1_strikes_a,...,f2_r5_kds,f2_r5_sigstrikes_l,f2_r5_sigstrikes_a,f2_r5_strikes_l,f2_r5_strikes_a,f2_r5_tds_l,f2_r5_tds_a,f2_r5_subs_a,f2_r5_passes,f2_r5_revs
0,Raphael Assuncao,"Marlon Moraes ""Magic""",Raphael Assuncao,http://ufcstats.com/fight-details/4e8b2357fea2...,2019-02-02,0,2,12,3,13,...,None,None,None,None,None,None,None,None,None,None
1,Jose Aldo,"Renato Moicano ""Moicano""",Jose Aldo,http://ufcstats.com/fight-details/300f1489174a...,2019-02-02,0,10,27,10,27,...,None,None,None,None,None,None,None,None,None,None
2,Demian Maia,"Lyman Good ""Cyborg""",Demian Maia,http://ufcstats.com/fight-details/ce246b336b6b...,2019-02-02,0,0,1,8,9,...,None,None,None,None,None,None,None,None,None,None
3,"Charles Oliveira ""Do Bronx""",David Teymur,"Charles Oliveira ""Do Bronx""",http://ufcstats.com/fight-details/85036b967403...,2019-02-02,0,15,38,18,41,...,None,None,None,None,None,None,None,None,None,None
4,Johnny Walker,"Justin Ledet ""El Blanco""",Johnny Walker,http://ufcstats.com/fight-details/44cfd4330eb6...,2019-02-02,1,7,8,7,8,...,None,None,None,None,None,None,None,None,None,None
5,"Livinha Souza ""The Brazilian Gangsta""","Sarah Frota ""A Treta""","Livinha Souza ""The Brazilian Gangsta""",http://ufcstats.com/fight-details/53abea410a05...,2019-02-02,0,11,16,28,33,...,None,None,None,None,None,None,None,None,None,None
6,"Markus Perez ""Maluko""","Anthony Hernandez ""Fluffy""","Markus Perez ""Maluko""",http://ufcstats.com/fight-details/ebb685a81f65...,2019-02-02,0,15,30,16,31,...,None,None,None,None,None,None,None,None,None,None
7,"Mara Romero Borella ""Kunoichici""",Taila Santos,"Mara Romero Borella ""Kunoichici""",http://ufcstats.com/fight-details/904e0e46392d...,2019-02-02,0,11,17,36,42,...,None,None,None,None,None,None,None,None,None,None
8,"Thiago Alves ""Pitbull""","Max Griffin ""Pain""","Thiago Alves ""Pitbull""",http://ufcstats.com/fight-details/59097097bc09...,2019-02-02,0,29,53,29,53,...,None,None,None,None,None,None,None,None,None,None
9,"Junior Albini ""Baby""","Jairzinho Rozenstruik ""Bigi Boy""","Junior Albini ""Baby""",http://ufcstats.com/fight-details/491b7d974254...,2019-02-02,0,12,21,20,29,...,None,None,None,None,None,None,None,None,None,None


In [9]:
# Save as csv
all_stats_df.to_csv("full_stats.csv")

## Testing for Event Page

In [3]:
te_url = 'http://ufcstats.com/event-details/84283233ec42be5f'
te_page = requests.get(te_url)
te_soup = BeautifulSoup(te_page.content, 'html.parser')

text = te_soup.find("li",{"class":"b-list__box-list-item"}).text.strip().strip("Date:").strip("\n").strip()

In [78]:
text

'February 02, 2019'

In [79]:
parser.parse(text)

datetime.datetime(2019, 2, 2, 0, 0)

### Testing for Fight Page

In [34]:
tf_url = 'http://ufcstats.com/fight-details/904e0e46392dff93'
tf_page = requests.get(tf_url)
tf_soup = BeautifulSoup(tf_page.content,'html.parser')

In [35]:
fs_table = tf_soup.findAll("table",{"class","b-fight-details__table js-fight-table"}) ## Start here
table_0 = fs_table[0]

In [36]:
p_ele = table_0.findAll('p',{'class':"b-fight-details__table-text"})

In [48]:
print(p_ele[14].text.replace('\n','').strip()) # Sub attempt f1
print(p_ele[16].text.replace('\n','').strip()) # Pass attempt f1
print(p_ele[18].text.replace('\n','').strip()) # Reversal attempt f1

0
2
0


In [57]:
p_ele

[<p class="b-fight-details__table-text">
 <a href="http://ufcstats.com/fighter-details/87f19d19f4cb73f8" style="color:#B10101">Magomed Bibulatov </a>
 </p>, <p class="b-fight-details__table-text">
 <a href="http://ufcstats.com/fighter-details/07a2bdaa0afdd831" style="color:#B10101">Rogerio Bontorin </a>
 </p>, <p class="b-fight-details__table-text">
       0
     </p>, <p class="b-fight-details__table-text">
       0
     </p>, <p class="b-fight-details__table-text">
       14 of 24
     </p>, <p class="b-fight-details__table-text">
       13 of 20
     </p>, <p class="b-fight-details__table-text">
       58%
     </p>, <p class="b-fight-details__table-text">
       65%
     </p>, <p class="b-fight-details__table-text">
       16 of 26
     </p>, <p class="b-fight-details__table-text">
       23 of 30
     </p>, <p class="b-fight-details__table-text">
       3 of 4
     </p>, <p class="b-fight-details__table-text">
       0 of 0
     </p>, <p class="b-fight-details__table-text">
      

In [9]:
f1_stats['Sig. str.'][0].split()

KeyError: 'Sig. str.'